# Angry Tweets - analiza Tweetów z wykorzystaniem Pythona

Analizie poddany został zbiór danych zawierający kilka tysięcy Tweetów.

Dane zawierają informacje o treści danego Tweeta oraz przyporządkowanie do konkretnej kategorii Tweeta: pozytywnej, negatywnej bądź neutralnej.

Na bazie posiadanych informacji stworzono algorytm, w wyniku działania którego otrzymujemy Tweety zaklasyfikowane do odpowiednich kategorii.

## 1. Importowanie bibliotek

W celu wykonania zadania, konieczne jest zaimportowanie odpowiednich bibliotek pozwalających odpowiednio przetworzyć dane.

In [2]:
import sys
import os
import time
import pandas as pd
import numpy as np
import re
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report

## 2. Definiowanie zmiennych

Do zmiennej *tweets_set* przypisujemy zawartość pliku train.csv korzystając z funkcji *read_csv* z pakietu *pandas*.

Zmienna *classes* jest listą kategorii, do których może zostać zakwalifykowany Tweet.

Zmienna *stop_words* jest listą słów z języka angielskiego, które nie będę uwaględnienie w procesie klasyfikacji Tweetów ze względu na zerowy wpływ na przyporządkowanie Tweeta do danej kategorii.

Korzystając z metody *random.rand* z pakietu *numpy* dzielimy wczytany zbiór Tweetów na dane treningowe oraz testowe w stosunku 70% treningowe - 30% testowe.

Dodatkowe tworzymy puste listy dla Tweetów testowych jak i treningowych, odpowiednio dla ich wartości tekstowej (Data) i kategorii (Label).

In [8]:
tweets_set= pd.read_csv("train.csv", sep=',', header = 1)

classes = ['negative', 'positive', 'neutral']
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

temp = np.random.rand(len(tweets_set)) < 0.80
tweetsTrain = tweets_set[temp]
tweetsTest = tweets_set[~temp]

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestData = []
tweetsTestLabel = []

## 3. Przetwarzanie Tweetów

Dla każdego wiersza w liście treningowej/testowej Tweetów wykonujemy następujace operacje:

Korzystając z funkcji *sub* z biblioteki *re* usuwamy z treści Tweetów wszystkie słowa rozpoczynające się od symbolu @.
Używa się go, gdy chce oznaczyć w Tweecie jakiegoś użytkownika, co do procesu klasyfikacji Tweetów nie dostarcza żadnej wartościowej informacji.

W następnym kroku analizujemy zawartość Tweeta po wykorzystaniu funkcji *split*, by móc wyszukać w niej słowa występujące w *stop_words*.

Po eweuntualnym usunięciu zbędnych słów łączymy zawartość tekstową Tweeta w całość i dodajemy do list *TrainData* i *TrainLabel* odpowiednio zawartość Tweeta i jego klasyfikację.

In [9]:
for index, tweet in tweetsTrain.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTrainData.append(tweet[2])
    tweetsTrainLabel.append(tweet[1])

for index, tweet in tweetsTest.iterrows():
    tweet[2] = re.sub(r'@\w+', '', tweet[2])
    querywords = tweet[2].split()
    resultwords  = [word for word in querywords if word.lower() not in stop_words]
    tweet[2] = ' '.join(resultwords)
    tweetsTestData.append(tweet[2])
    tweetsTestLabel.append(tweet[1])

## 4. Reprezentacja danych

Przygotowując dane do klasyfikacji, musimi posiadać obiekty są reprezentowane przez ich cechy.

W przypadku Tweetów, tymi cechami są występujące w nich wyrazy.

By móc otrzymać racjonalne wyniki klasyfikacji, należu nadać odpowiednie wagi słowom ze wględu na częstotliwość ich występowania w Tweetach.

Scikit-learn udostępnia metodę TfidfVectorizer, która transformuje dane wejściowe w vectory nadające się do użycia z klasyfikatorami.

Zmienna dla metody *TfidfVectorizer* ustawiamy parametry:

   *min_df* - odrzucane są słowa występujące w mniej niż 15 Tweetach,

   *max_df* - odrzuca słowa pojawiające się w ponad 80% dokumentów,

   *sublinear_tf* i *use_idf* - używanie *sublinear weighting* i *wagowania IDF*

Funkcja *fit_transform* tworzy słownik słowo-waga.

Funkcja *transform* przemiania dane testowe do odpowiadającej formy słownikowej.

In [10]:
vectorizer = TfidfVectorizer(min_df = 10,
                             max_df = 0.80,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

## 5. Klasyfikacja

Scikit-learn udostępnia wiele klasyfikatorów. Bo dokonaniu dogłębnych testów, do tego zagadnienia wybrane zostały trzy:

*MultinomialNB* - Naiwny Bayes,
*LogisticRegression* - Regresja Logistyczna,
*LinearSVC* - Linear Support Vector Classification.



In [11]:
classifierNB = MultinomialNB()
classifierNB.fit(train_vectors, tweetsTrainLabel)
predictionNB=classifierNB.predict(test_vectors)

classifierLog = LogisticRegression(penalty='l1')
classifierLog.fit(train_vectors, tweetsTrainLabel)
predictionLog=classifierLog.predict(test_vectors)

classifierLSVC = svm.LinearSVC()
classifierLSVC.fit(train_vectors, tweetsTrainLabel)
predictionLSVC = classifierLSVC.predict(test_vectors)

In [12]:
print("Results for MultinomialNB()")
print(classification_report(tweetsTestLabel, predictionNB))
print("Results for LogisticRegression()")
print(classification_report(tweetsTestLabel, predictionLog))
print("Results for LinearSVC()")
print(classification_report(tweetsTestLabel, predictionLSVC))

Results for MultinomialNB()
             precision    recall  f1-score   support

   negative       0.54      0.10      0.17       193
    neutral       0.51      0.37      0.43       445
   positive       0.58      0.86      0.69       574

avg / total       0.55      0.56      0.51      1212

Results for LogisticRegression()
             precision    recall  f1-score   support

   negative       0.39      0.16      0.22       193
    neutral       0.46      0.35      0.40       445
   positive       0.58      0.81      0.68       574

avg / total       0.51      0.54      0.50      1212

Results for LinearSVC()
             precision    recall  f1-score   support

   negative       0.40      0.25      0.31       193
    neutral       0.47      0.38      0.42       445
   positive       0.59      0.75      0.66       574

avg / total       0.51      0.53      0.51      1212

